# Classification on CIFAR and ImageNet

In [ ]:
import os
import time
import json
import copy
from pathlib import Path
import datetime

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import models
import ops.trains as trains
import ops.tests as tests
import ops.datasets as datasets
import ops.schedulers as schedulers

In [ ]:
# config_path = "configs/cifar10_general.json"
config_path = "configs/cifar100_general.json"
# config_path = "configs/imagenet_general.json"

with open(config_path) as f:
    args = json.load(f)
    print("args: \n", args)

In [ ]:
dataset_args = copy.deepcopy(args).get("dataset")
train_args = copy.deepcopy(args).get("train")
val_args = copy.deepcopy(args).get("val")
model_args = copy.deepcopy(args).get("model")
optim_args = copy.deepcopy(args).get("optim")
env_args = copy.deepcopy(args).get("env")

In [ ]:
dataset_train, dataset_test = datasets.get_dataset(**dataset_args, download=True)
dataset_name = dataset_args["name"]
num_classes = len(dataset_train.classes)

dataset_train = DataLoader(dataset_train, 
                           shuffle=True, 
                           num_workers=train_args.get("num_workers", 4), 
                           batch_size=train_args.get("batch_size", 128))
dataset_test = DataLoader(dataset_test, 
                          num_workers=val_args.get("num_workers", 4), 
                          batch_size=val_args.get("batch_size", 128))

print("Train: %s, Test: %s, Classes: %s" % (
    len(dataset_train.dataset), 
    len(dataset_test.dataset), 
    num_classes
))

## Model

In [ ]:
# AlexNet
# name = "alexnet_dnn"
# name = "alexnet_mcdo"
# name = "alexnet_dnn_smoothing"
# name = "alexnet_mcdo_smoothing"

# VGG
# name = "vgg_dnn_16"
# name = "vgg_mcdo_16"
# name = "vgg_dnn_smoothing_16"
# name = "vgg_mcdo_smoothing_16"

# Preact VGG
# name = "prevgg_dnn_16"
# name = "prevgg_mcdo_16"
# name = "prevgg_dnn_smoothing_16"
# name = "prevgg_mcdo_smoothing_16"

# ResNet
name = "resnet_dnn_18"
# name = "resnet_mcdo_18"
# name = "resnet_dnn_smoothing_18"
# name = "resnet_mcdo_smoothing_18"

# name = "resnet_dnn_50"
# name = "resnet_mcdo_50"
# name = "resnet_dnn_smoothing_50"
# name = "resnet_mcdo_smoothing_50"

# Preact ResNet
# name = "preresnet_dnn_50"
# name = "preresnet_mcdo_50"
# name = "preresnet_dnn_smoothing_50"
# name = "preresnet_mcdo_smoothing_50"

# ResNeXt
# name = "resnext_dnn_50"
# name = "resnext_mcdo_50"
# name = "resnext_dnn_smoothing_50"
# name = "resnext_mcdo_smoothing_50"

# WideResNet
# name = "wideresnet_dnn_50"
# name = "wideresnet_mcdo_50"
# name = "wideresnet_dnn_smoothing_50"
# name = "wideresnet_mcdo_smoothing_50"


model = models.get_model(name, num_classes=num_classes, 
                         stem=model_args.get("stem", False))
# models.load(model, dataset_name, uid=current_time)

Parallelize the given `moodel` by splitting the input:

In [ ]:
name = model.name
model = nn.DataParallel(model)
model.name = name

## Train

Define a TensorBoard writer:

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
log_dir = os.path.join("runs", dataset_name, model.name, current_time)
writer = SummaryWriter(log_dir)

with open("%s/config.json" % log_dir, "w") as f:
    json.dump(args, f)
with open("%s/model.log" % log_dir, "w") as f:
    f.write(repr(model))

print("Create TensorBoard log dir: ", log_dir)

Train the model:

In [ ]:
gpu = torch.cuda.is_available()
optimizer, train_scheduler = trains.get_optimizer(model, **optim_args)
warmup_scheduler = schedulers.WarmupScheduler(optimizer, len(dataset_train) * train_args.get("warmup_epochs", 0))

trains.train(model, optimizer,
             dataset_train, dataset_test,
             train_scheduler, warmup_scheduler,
             train_args, val_args, gpu,
             writer)

Save the model:

In [ ]:
models.save(model, dataset_name, current_time, optimizer=optimizer)

## Test

In [ ]:
gpu = torch.cuda.is_available()

model = model.cuda() if gpu else model.cpu()
metrics_list = []
for n_ff in [1]:
    print("N: %s, " % n_ff, end="")
    *metrics, cal_diag = tests.test(model, n_ff, dataset_test, verbose=False, gpu=gpu)
    metrics_list.append([n_ff, *metrics])

leaderboard_path = os.path.join("leaderboard", "logs", dataset_name, model.name)
Path(leaderboard_path).mkdir(parents=True, exist_ok=True)
metrics_dir = os.path.join(leaderboard_path, "%s_%s_%s.csv" % (dataset_name, model.name, current_time))
tests.save_metrics(metrics_dir, metrics_list)

In [ ]:
gpu = torch.cuda.is_available()

model = model.cuda() if gpu else model.cpu()
metrics_list = []
for n_ff in [1, 2, 3, 4, 5, 10, 20, 50]:
    print("N: %s, " % n_ff, end="")
    *metrics, cal_diag = tests.test(model, n_ff, dataset_test, verbose=False, gpu=gpu)
    metrics_list.append([n_ff, *metrics])

leaderboard_path = os.path.join("leaderboard", "logs", dataset_name, model.name)
Path(leaderboard_path).mkdir(parents=True, exist_ok=True)
metrics_dir = os.path.join(leaderboard_path, "%s_%s_%s.csv" % (dataset_name, model.name, current_time))
tests.save_metrics(metrics_dir, metrics_list)